# Initialization

In [ ]:
%matplotlib inline
import os, sys
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

from time import sleep, monotonic, time
import datetime as dt

import qcodes as qc
from qcodes import Parameter
from qcodes import initialise_or_create_database_at
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.utils.metadata import diff_param_values
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.measurements import Measurement

sys.path.append("C:/Users/TUD210595/Documents/LOCAL_Qcodes/scripts")
from sweeps import do1d, do2d, time_sweep, measure_until, do1d_until

import warnings
warnings.filterwarnings('ignore')

To modify list of instruments, eddit file
"C:\Users\TUD210595\Documents\LOCAL_Qcodes\scripts\init_VectorJanis.py"

In [ ]:
fridge_name = 'VectorJanis'
%run C:\Users\TUD210595\Documents\LOCAL_Qcodes\scripts\init_fridge.py {fridge_name}

Here are all the instruments we have:

In [ ]:
station.components

Zero dacs and lockin

In [ ]:
ivvi.set_dacs_zero()
lockin_1.amplitude(0.004)

## DAC rates

Change the dac rate (same idea as with qtlab, but two commands are used)

In [ ]:
ivvi.dac1.step = 50
ivvi.dac1.inter_delay = 0.05
ivvi.dac2.step = 50
ivvi.dac2.inter_delay = 0.05
ivvi.dac3.step = 4
ivvi.dac3.inter_delay = 0.05

## Database initialisation

Next, we create a qcodes database, or we can use the existing one

In [ ]:
#useful if you want to name database with creation date
date = str(dt.date.today())
db_path = "C:/Users/TUD210595/Documents/LOCAL_DATA/Ivan"
initialise_or_create_database_at(db_path + "/" + "cool_down.db" )

# Scaled parameters

It is more convenient to use meaningful parameters with correct units

In [ ]:
from qcodes import ManualParameter, ScaledParameter

## Defining scaled parameters. Run ONLY ONCE!

In [ ]:
Irange = Parameter('Irange', label='current range', unit='', set_cmd=None, get_cmd=None)
Vrange = Parameter('Vrange', label='voltage range', unit='', set_cmd=None, get_cmd=None)
TGrange = Parameter('TGrange', label='Top gate range', unit='', set_cmd=None, get_cmd=None)

#division paraemtrs required for simplier update: they used as an argument for scaled paraemtrs of sorce instruments

Irange_division = Parameter('Irange_division', 
                            label='scaled current range', unit='', 
                            set_cmd=None, get_cmd=lambda : 1000 / Irange())

Vrange_division = Parameter('Vrange_division', 
                            label='scaled voltage range', unit='', 
                            set_cmd=None, get_cmd=lambda : 1000 / Vrange())

TGrange_division = Parameter('TGrange_division', 
                             label='scaled Top gate range', unit='', 
                             set_cmd=None, get_cmd=lambda : 1000 / TGrange())

Irange_AC_division = Parameter('Irange_AC_division', 
                               label='scaled current range for lockin', unit='', 
                               set_cmd=None, get_cmd=lambda : 100 / Irange())

Vrange_AC_division = Parameter('Vrange_AC_division', 
                               label='scaled voltage range for lockin', unit='', 
                               set_cmd=None, get_cmd=lambda : 100 / Vrange())

Gain_K1 = Parameter('Gain_K1', label='Gain K1', unit='', set_cmd=None, get_cmd=None)
Gain_K2 = Parameter('Gain_K2', label='Gain K2', unit='', set_cmd=None, get_cmd=None)
Gain_L1 = Parameter('Gain_L1', label='Gain L1', unit='', set_cmd=None, get_cmd=None)
# Gain_L2 = Parameter('Gain_L2', label='Gain L2', unit='', set_cmd=None, get_cmd=None)

#Add components
station.add_component(Irange)
station.add_component(Vrange)
station.add_component(TGrange)

station.add_component(Irange_division)
station.add_component(Vrange_division)
station.add_component(TGrange_division)

station.add_component(Irange_AC_division)
station.add_component(Vrange_AC_division)
                             
station.add_component(Gain_K1)
station.add_component(Gain_K2)
station.add_component(Gain_L1)
# station.add_component(Gain_L2)

## IVVI rack parameters - UPDATE here DURING measuerements

Also can be coppied to execute later

In [ ]:
Irange.set(100e-9)
Vrange.set(1e-3)
TGrange.set(15)
Gain_K1.set(1e2)
Gain_K2.set(1e2)
# Gain_K3.set(1e2)
Gain_L1.set(Gain_K1())
# Gain_L2.set(Gain_K2())

#division parameters are updated automatically

## DC parameters - assigning devices. RUN ONLY ONCE!
Check that devicies are correct!

In [ ]:
appl_current = ScaledParameter(ivvi.dac1, division = Irange_division, name = 'appl_current', unit = 'A')
appl_voltage = ScaledParameter(ivvi.dac2, division = Vrange_division, name = 'appl_voltage', unit = 'V')
appl_TG = ScaledParameter(ivvi.dac3, division = TGrange_division, name = 'appl_TG', unit = 'V')

meas_voltage_K1 = ScaledParameter(keithley_1.amplitude, division = Gain_K1, name = 'meas_voltage_K1', unit = 'V')
meas_voltage_K2 = ScaledParameter(keithley_2.amplitude, division = Gain_K2, name = 'meas_voltage_K2', unit = 'V')
# meas_voltage_K3 = ScaledParameter(keithley_3.amplitude, division = Gain_K3, name = 'meas_voltage_K3', unit = 'V')
meas_leakage = ScaledParameter(keithley_2.amplitude, division = Gain_K2, name = 'meas_leakage', unit = 'A')

meas_current_K1 = ScaledParameter(keithley_1.amplitude, division = Gain_K1, name = 'meas_current_K1', unit = 'A')
meas_current_K2 = ScaledParameter(keithley_2.amplitude, division = Gain_K2, name = 'meas_current_K2', unit = 'A')
# meas_current_K3 = ScaledParameter(keithley_3.amplitude, division = Gain_K3, name = 'meas_current_K3', unit = 'A')

station.add_component(appl_current)
station.add_component(appl_voltage)
station.add_component(appl_TG)
station.add_component(meas_voltage_K1)
station.add_component(meas_voltage_K2)
# station.add_component(meas_voltage_K3)
station.add_component(meas_leakage)
station.add_component(meas_current_K1)
station.add_component(meas_current_K2)
# station.add_component(meas_current_K3)

Reasining devices seems to be missing in public methods of ScaledParameter class... But you can try:

In [ ]:
# appl_current._wrapped_parameter = ivvi.dac1

## AC parameters

In [ ]:
appl_current_AC = ScaledParameter(lockin_1.amplitude, division = Irange_AC_division, name = 'appl_current_AC', unit = 'A')
appl_voltage_AC = ScaledParameter(lockin_1.amplitude, division = Vrange_AC_division, name = 'appl_voltage_AC', unit = 'V')

station.add_component(appl_current_AC)
station.add_component(appl_voltage_AC)

meas_voltage_AC_L1 = ScaledParameter(lockin_1.X, division = Gain_L1, name = 'meas_voltage_Lockin1', unit = 'V')

meas_current_AC_L1 = ScaledParameter(lockin_1.X, division = Gain_L1, name = 'meas_current_Lockin1', unit = 'A')

station.add_component(meas_voltage_AC_L1)
station.add_component(meas_current_AC_L1)

## Another useful example: magnet using dac (Janis)

>2231.2 G per A => 0.22312 T per A<br>
I source 10mA per V => 1e-5 A per 1mV<br>
result: 0.22312 T/A * 1e-5 A/mV

In [ ]:
#magnetic_field = ScaledParameter(ivvi.dac3, gain = 0.22312e-5, name = 'magnetic_field_dac', unit = 'T')
#station.add_component(magnetic_field)

# Fidge controll. Condensing. Magnet

In [ ]:
ivvi.set_dacs_zero()
lockin_1.amplitude(0.004)

In [ ]:
lakeshore.output_1.output_range('off')
# lakeshore.output_1.setpoint(45.0)
# lakeshore.output_1.input_channel('A')
# lakeshore.output_1.output_range('2.5W')

In [ ]:
print('Temps: A= {:.2f}, B = {:.4f}, C ={:.4f}'.format(lakeshore.A.temperature(), lakeshore.B.temperature(), lakeshore.C.temperature()) )

In [ ]:
level_meter.print_readable_snapshot(update=True)

In [ ]:
# level_meter.sample_mode('Continuous')
# level_meter.sample_mode('Sample/Hold')

In [ ]:
while True:
    print('A= {:.2f}, B = {:.4f}, C ={:.4f}, power = {:.2f}'.format(lakeshore.A.temperature(), lakeshore.B.temperature(), lakeshore.C.temperature(), lakeshore.output_1.output()), end = '\r')
    sleep(1)

In [ ]:
while True:
    print('He Level = {:.2f}'.format(level_meter.lastval()), end = '\r')
    sleep(1)

In [ ]:
cryomag.print_readable_snapshot(update = True)

In [ ]:
cryomag.channel_A.heater('off')
cryomag.channel_B.heater('off')

In [ ]:
print('feild 4T magnet = {:.4f} T'.format(cryomag.channel_A.field()) )
print('tolerance for set field 4T magnet = {:.1f} mT'.format(cryomag.channel_A.tolerance()) )

print('feild 9T magnet = {:.4f} T'.format(cryomag.channel_B.field()) )
print('tolerance for set field 9T magnet = {:.1f} mT'.format(cryomag.channel_B.tolerance()) )

# Standard sweeps: measuring without setting value

## Measured values versus non-setable parameter

In [ ]:
time = ElapsedTimeParameter('time')

def time_limit(time, measured_values):
    return time() > 6

In [ ]:
exp = qc.new_experiment('V_vs_time', sample_name='simulator_100_Ohm')
current(10e-6)
current_AC(1e-6)
time.reset_clock()
dataid = measure_until(time, time_limit, 1, voltage_DC, voltage_AC)

In [ ]:
plot_by_id(dataid)

## Useful for temperature scans and magnet sweeps:

In [ ]:
keithley_1.amplitude()

In [ ]:
time = ElapsedTimeParameter('time')

def time_limit(time, measured_values):
    return time() > 7 * 60 * 60

In [ ]:
exp = qc.new_experiment('R_vs_time', sample_name='Fridge')
time.reset_clock()
dataid = measure_until(time, time_limit, 10, keithley_1.amplitude)

In [ ]:
def temp_limit(temp, measured_values):
    return temp() <

exp = qc.new_experiment('V_vs_temp_CD', sample_name='simulator_100_Ohm')
current(10e-6)
current_AC(1e-6)
dataid = measure_until(lakeshore.A.temperature, temp_limit, 1, voltage_DC, voltage_AC)

In [ ]:
def resistance_limit(time, measured_values):
    return measured_values[0].get() > -0.94e-3

exp = qc.new_experiment('V_vs_time', sample_name='simulator_100_Ohm')
current(10e-6)
current_AC(1e-6)
time = ElapsedTimeParameter('time')
time.reset_clock()
dataid = measure_until(time, voltage_limit, 1, voltage_DC, voltage_AC)

In [ ]:
plot_by_id(dataid)